In [1]:
import numpy as np

from sklearn.metrics import accuracy_score

In [2]:
!pip install tensorflow
!pip install gluoncv

You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.
  Using cached gluoncv-0.10.4.post4-py2.py3-none-any.whl (1.3 MB)
  Using cached autocfg-0.0.8-py3-none-any.whl (13 kB)
  Using cached portalocker-2.3.2-py2.py3-none-any.whl (15 kB)
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
     |████████████████████████████████| 76 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 197 kB 5.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
class_names = {'plane':0, 'car':1, 'bird':2, 'cat':3, 'deer':4,
               'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}

In [4]:
from experiments.util import load_test

X_test, y_test = load_test()

#### Use pretrained model (on CIFAR 10)

##### ResNet110

In [5]:
from mxnet.gluon.data.vision import transforms

import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = [mx.cpu(0)]

# Get the model CIFAR_ResNet20_v1, with 10 output classes, without pre-trained weights
net = get_model('cifar_resnet110_v2', pretrained=True, classes=10)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

y_test_hat_prob = net(transform_test(mx.ndarray.array(X_test)))

y_test_hat = np.argmax(y_test_hat_prob.asnumpy(), axis=1)

y_test_enc = np.vectorize(class_names.get)(y_test)

print(accuracy_score(y_test_hat, y_test_enc))

0.9554
